### 경기 남부 경찰청

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
#메인 주소 : https://www.ggpolice.go.kr/main/html.do?menu=HC44
# https://www.ggpolice.go.kr/swjb/html.do?menu=HC24

In [3]:
base_url = 'https://www.ggpolice.go.kr'
main_url = '/main/html.do?menu=HC44'
url = f'{base_url}{main_url}'
result = requests.get(url)
soup = BeautifulSoup(result.text)

In [4]:
lis = soup.select('li.road')
len(lis)

31

In [5]:
li = lis[0]

In [6]:
href = li.select_one('.road > a')['href'][26:]
href

'/swjb/html.do?menu=HC24'

In [7]:
sub_url = f'{base_url}{href}'
res = requests.get(sub_url)
road_soup = BeautifulSoup(res.text)

In [8]:
info = road_soup.select_one('.cont_in')
info

<div class="cont_in">
<!-- * 카카오맵 - 지도퍼가기 --><!-- 1. 지도 노드 -->
<div class="root_daum_roughmap root_daum_roughmap_landing" id="daumRoughmapContainer1649900046443"></div>
<!--
	2. 설치 스크립트
	* 지도 퍼가기 서비스를 2개 이상 넣을 경우, 설치 스크립트는 하나만 삽입합니다.
--><script charset="UTF-8" class="daum_roughmap_loader_script" src="https://ssl.daumcdn.net/dmaps/map_js_init/roughmapLoader.js"></script><!-- 3. 실행 스크립트 --><script charset="UTF-8">
	new daum.roughmap.Lander({
		"timestamp" : "1649900046443",
		"key" : "29txp",
		"mapWidth" : "850",
		"mapHeight" : "580"
	}).render();
</script>
<ul class="basic_list7">
<li>기관명 : 수원중부경찰서</li>
<li>도로명 주소 : 경기도 수원시 장안구 정자천로 199 (우)16335</li>
<li>대표전화 : 경찰민원 콜센터 182</li>
</ul>
<div class="align_right mgnb30"><a href="http://map.naver.com/?dlevel=12&amp;pinType=site&amp;pinId=11642405&amp;x=126.9962821&amp;y=37.2979559&amp;enc=b64" target="_blank"><img alt="네이버 지도검색" src="/include/police/images/cont/sub4_3_in02.jpg"> </img></a> <a href="http://map.daum.net/?urlX=499177&amp;urlY

In [9]:
name = info.select('.basic_list7 > li')[0].text[6:]
name

'수원중부경찰서'

In [10]:
addr = info.select('.basic_list7 > li')[1].text[9:-9]
addr

'경기도 수원시 장안구 정자천로 199'

In [11]:
s_police = []
for li in lis:
    href = li.select_one('.road > a')['href'][26:]
    res = requests.get(base_url + href)
    road_soup = BeautifulSoup(res.text)
    info = road_soup.select_one('.cont_in')
    try:
        name = info.select('.basic_list7 > li')[0].text[6:]
        addr = info.select('.basic_list7 > li')[1].text[9:-9]
    except:
        continue
    s_police.append({'기관명' :name, '주소': addr})

In [12]:
s_police = pd.DataFrame(s_police)
s_police

,기관명,주소
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199
1,수원남부경찰서,경기 수원시 영통구 매봉로 52
2,수원서부경찰서,경기 수원시 권선구 서부로 1673
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63
5,과천경찰서,경기 과천시 통영로 20
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10
8,분당경찰서,경기 성남시 분당구 정자일로 165
9,부천소사경찰서,경기 부천시 부광로 214


In [13]:
import folium
import pandas as pd
import requests, json
from urllib.parse import quote
import fontawesome as fa

In [14]:
with open('../04.지도시각화/data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [15]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [16]:
lat_list, lng_list = [], []
for i in s_police.index:
    url = f'{base_url}?query={quote(s_police.주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [17]:
import random
colors = ['pink', 'lightgreen', 'darkgreen', 'black', 'lightred', 'darkblue', 'lightblue',
          'purple', 'green', 'cadetblue', 'gray', 'red', 'blue', 'lightgray', 'darkpurple', 'orange', 'darkred', 'beige']
random.choice(colors)

'darkred'

In [18]:
s_police['위도'] = lat_list
s_police['경도'] = lng_list
color_list = []
for i in s_police.index:
    color_list.append(random.choice(colors))
s_police['color'] = color_list
s_police

,기관명,주소,위도,경도,color
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199,37.297932,126.996317,lightblue
1,수원남부경찰서,경기 수원시 영통구 매봉로 52,37.272104,127.054492,green
2,수원서부경찰서,경기 수원시 권선구 서부로 1673,37.258740,126.972264,lightred
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43,37.391209,126.948866,darkred
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63,37.387334,126.925974,darkblue
5,과천경찰서,경기 과천시 통영로 20,37.429358,126.989987,darkblue
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259,37.442287,127.126662,black
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10,37.442664,127.170371,orange
8,분당경찰서,경기 성남시 분당구 정자일로 165,37.365305,127.105450,red
9,부천소사경찰서,경기 부천시 부광로 214,37.468723,126.824181,gray


In [21]:
map = folium.Map(location=[s_police.위도.mean(), s_police.경도.mean()], zoom_start=10)
for i in s_police.index:
    folium.Marker(
        location=[s_police.위도[i], s_police.경도[i]],
        popup=folium.Popup(s_police.주소[i], max_width=200),
        tooltip=s_police.기관명[i],
        icon=folium.Icon(color = s_police.color[i])
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">수원남부경찰서</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

- 오류 해결
    - 어디서 발생했는지 파악
    - 파악한 곳으로 가서 해결

In [ ]:
# 어디서 에러가 발생했는지 파악
lines = []
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        print(index)

In [ ]:
# 에러를 찾아서 해결
# index=6, 군포경찰서로 찾아가서 원인 파악
li = lis[6]
href = li.find('a')['href']
print(href)
sub_res = requests.get(href)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
sub_lis = sub_soup.select('.basic_list7 > li')
print(len(sub_lis))

In [ ]:
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_res = requests.get(href)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7 > li')
    if len(sub_lis) == 0:
        continue
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'name':name, 'addr':addr})

In [ ]:
# 정확한 원인을 찾지 못하는 경우, just skip
lines = []
for li in lis:
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        continue

In [ ]:
# 중복 여부 확인
len(df.name.unique())

In [ ]:
# 중복 제거
df.drop_duplicates(subset='name', inplace=True)
df.shape